In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from matplotlib.colors import ListedColormap

In [7]:
#load data

def load_data(name):
    df = pd.read_csv(name + '.csv')
    #df.dob=pd.to_datetime(df.dob)
    
    return df

fifa_20 = load_data('data\\NBA_Player_Stats')
fifa_20.head()

,Rk,Player,Pos,Age,PTS,AST,STL,BLK,Tm,G,GS,MP
0,1,Precious Achiuwa,C,22,9.1,1.1,0.5,0.6,TOR,73,28,23.6
1,2,Steven Adams,C,28,6.9,3.4,0.9,0.8,MEM,76,75,26.3
2,3,Bam Adebayo,C,24,19.1,3.4,1.4,0.8,MIA,56,56,32.6
3,4,Santi Aldama,PF,21,4.1,0.7,0.2,0.3,MEM,32,0,11.3
4,5,LaMarcus Aldridge,C,36,12.9,0.9,0.3,1.0,BRK,47,12,22.3


In [17]:
# create player main position
def create_main_position(data):
        data.loc[:, 'main_position'] = data['Pos'].apply(lambda x: x.split(',')[0])


create_main_position(fifa_20)

In [24]:
target_pos = ['C', 'SG', 'SF']
b = 100000000
n_pop = 805
prob_mutation = 0.5
prob_crossover = 0.8
elitism_size = 0.6
test_team = 'Phoenix'


def init_population(data, n, positions_target):

    population = []
    for _ in range(n):
        indv = []
        for j in positions_target:
            p = data[data['main_position'] == j].sample(1)
            indv.append(p.iloc[0].Pos)
        population.append(indv)

    return population
    



#One point Crossover
def crossover(indv_1, indv_2):

    if random.random() < prob_crossover:

        pos = random.randint(0,len(indv_1))
        f1 = indv_1[:pos] + indv_2[pos:]
        f2 = indv_2[:pos] + indv_1[pos:]
        return [f1, f2]

    else:
        return [indv_1,indv_2]


def random_mutation(indv, positions, data):
    new_indv = indv.copy()

    for i in range(len(indv)):
        if  random.random() < prob_mutation:
            p = data[data['main_position'] == positions[i]].sample(1)

            new_indv[i] = p.iloc[0]['Player']

    return new_indv


def evaluate_population(original_data, population, budget):
    alpha = 0.6
    beta = 0
    gamma = 0
    theta = 0.4
    
    max_values = [budget, 35, 7, 10550000]
    
    scores = []
    for indv in population:

        accumulated_value = []
        ages = []
        team_players = []
        for i in indv:
            p = original_data[original_data['Player'] == i].iloc[0]
            
            accumulated_value.append(1.2 * p.value_eur)
            ages.append(p.age)
            team_players.append(len(original_data[(original_data['team'] == test_team) & (original_data['Pos'] == p.main_position)]))
                
        scores.append(round(alpha * (np.abs(budget - np.sum(accumulated_value))/max_values[0])
                            + beta * (np.median(ages)/max_values[1]) 
                            + gamma * (np.mean(team_players)/max_values[2])
                            + theta * (10550000-np.min(accumulated_value)/max_values[3]), 2))

    return scores

def indv_value(indv, data):

    accumulated_value = 0
    for i in indv:
        p = data[data['Player'] == i]
        accumulated_value += p.iloc[0].value_eur
        
    return accumulated_value

def argsort_list(seq):
    return [x for x,y in sorted(enumerate(seq), key = lambda x: x[1])]


def elitism(original_data, population, offspring, budget):
    
    scores_pop = evaluate_population(original_data, population, budget)
    scores_offs = evaluate_population(original_data, offspring, budget)
    
    comp_elite = int(len(population) * elitism_size)
    
    population = [population[ind] for ind in argsort_list(scores_pop)]
    offspring = [offspring[ind] for ind in argsort_list(scores_offs)]
    
    new_population = population[:comp_elite] + offspring[:len(population) - comp_elite]

    return new_population


def evolve(data, target_positions, budget):

    num_iters = 50

    scores = np.zeros(num_iters)
    best_indv = None
    best_score = float('+inf')

    population = init_population(data, n_pop, target_positions)

    #print("Initial Population: ", population)
    
    for it in range(num_iters):

        pool = population
        
        #Crossover
        parents = []
        for i in  range(0, n_pop - 1, 2):
            indiv_1= pool[i]
            indiv_2 = pool[i+1]
            parents.extend(crossover(indiv_1, indiv_2)) 
        
        #Mutation
        offspring = []
        for indv in parents:
            offspring.append(random_mutation(indv, target_positions, data))
        
        #Select the best inviduals: Elitism
        population = elitism(fifa_20, population, offspring, budget)
        pop_scores = evaluate_population(data, population, budget)
                
        for i in range(len(pop_scores)):

            if pop_scores[i] < best_score:
                best_indv = pop_scores[i]
                best_players = population[i]
                best_score = pop_scores[i]

        scores[it] = np.mean(pop_scores)

        if it % 5 == 0:
            print("Iteration ", it, ": Average Score:", round(scores[it], 3), "Best so far:", round(best_score, 3), best_players)


    plt.figure(figsize=(12, 8))
    plt.plot(scores)
    plt.xlabel('Iterations')
    plt.ylabel('Score')
    plt.title('Score Evolution')
    plt.show()
        
    print('Best players for {}:'.format(target_positions))

    for i in best_players:
        p = data[data['Player'] == i]
        print('{}:\n\tAge: {}'.format(i, p.iloc[0].age))
            
    #print('\nMax Budget: ', budget)
    #print('Budget Used: ', indv_value(best_players, data))


    # CONFIGURATION PARAMETERS
evolve(fifa_20, target_pos, b)


IndexError: single positional indexer is out-of-bounds

For this specific configuration, the best set of player found were **[M. Sels, M. Skriniar, Andre Almeida and Jovane Cabral]**, for the positions **['GK', 'CB', 'RB', 'LW']**